# Jelszavak tárolása / storing passwords

In [ ]:
# Run this cell first
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
from Crypto.Hash import SHA256
import pandas as pd
import random, codecs

## Egyszerű hash tárolás / simple hash storage

In [ ]:
@interact
def gen_simple(password = "secret"):
    """This method demostrates simple password generation using SHA256"""
    
    h = SHA256.new()
    h.update(bytes(password,'utf-8'))
    return h.hexdigest()

In [ ]:
@interact
def verify_simple(user = 'simple', password = "secret"):
    """This method demostrates simple password verification using SHA256"""
    
    # challenge számítása / calculate challenge
    h = SHA256.new()
    h.update(bytes(password,'utf-8'))
    challenge = h.hexdigest()
    
    # adatbázis lekérdezés / database query
    db = pd.read_csv('userdb.csv')
    res = db[(db["user"] == user) & (db["password"] == challenge)]
  
    # végeredmény számítása / calculate final result
    if len(res) > 0: # van találat / we have a match
        return "authentication success - user and password match"
    else: # nincs találat / we do not have a match
        return "authentication failed - user or password mismatch"

## Sózott hash tárolás / salted hash storage

In [ ]:
@interact_manual
def gen_salted(password = "secret"):
    """This method demostrates salted password generation using SHA256"""
    
    salt = random.randrange(10000, 99999)
    
    h = SHA256.new()
    h.update(bytes(str(salt) + password,'utf-8'))
    return {"salt": str(salt), "hash": h.hexdigest()}

In [ ]:
@interact
def verify_salted(user = 'salted', password = "secret"):
    """This method demostrates salted password verification using SHA256"""
        
    # adatbázis lekérdezés / database query 
    db = pd.read_csv('userdb.csv')
    record = db[(db["user"] == user)]
    if len(record) < 1:
        return "authentication failed - no such user"
    stored = record["password"][1]
    salt = int(record["salt"][1])

    # challenge számítása / calculate challenge
    h = SHA256.new()
    h.update(bytes(str(salt) + password,'utf-8'))
    challenge = h.hexdigest()
    
    # végeredmény számítása / calculate final result
    if challenge == stored: # hash érték egyezik / hash value match
        return "authentication success - password match"
    else:
        return "authentication failed - password mismatch"

## Proof of work jelszótárolás (és borsozás) / proof of work storage (and peppering)

In [ ]:
@interact_manual
def gen_pow(password = "secret"):
    """This method demostrates proof of work password generation using SHA256"""
    
    salt = random.randrange(10000, 99999) # a só értékét tároljuk / salt is stored
    pepper = random.randrange(1, 9) # a bors értékét NEM tároljuk / pepper is NOT stored
    work_factor = 100000 # hash ismétlések száma, előre definiált / number of hash iterations, predefined
    
    h = SHA256.new()
    h.update(bytes(str(salt) + str(pepper) + password,'utf-8'))
    res = h.digest()
    
    for i in range(1, work_factor):
        h = SHA256.new()
        h.update(res)
        res = h.digest()        
    
    return {"salt": str(salt), "hash": bytes.decode(codecs.encode(res,'hex'))}

In [ ]:
%%timeit
x = gen_salted()

In [ ]:
%%timeit
x = gen_pow()

## SOHA nem gyártunk saját kirptográfiát! / NEVER roll your own crypto!

Az előző példák csupán az elvek bemutatását szolgálják. /
Previous examples are here only to show the principles.

In [ ]:
import bcrypt

@interact_manual
def gen_bcrypt(password = "secret"):
    """This method correct password generation using PBKDF2 and secure random hash"""

    salt = bcrypt.gensalt()
    hash = bcrypt.hashpw(password.encode('UTF-8'), salt)
    
    return {"salt": bytes.decode(salt), "hash": bytes.decode(hash)} 

In [ ]:
%%timeit
x = gen_bcrypt()

## Gyakorló feladat / practice task

Készítse el a bcrypt és a "proof of work" jelszó ellenőrző függvényt! /
Implement the bcrypt and the "proof of work" password verification method!